# Build MCP Tools with Bedrock AgentCore Gateway
Within Media and Entertainment organizations, many data sources/services are often siloed across different departments and platforms that could be leveraged by agentic applications. From content management systems and digital asset libraries to audience analytics platforms and distribution networks, these resources remain fragmented and difficult to access programmatically. A unified Model Context Protocol (MCP) gateway presents an ideal solution to this challenge by providing a standardized interface that seamlessly connects diverse media ecosystem components. Rather than requiring agentic applications to integrate with dozens of different APIs, authentication systems, and data formats, the MCP gateway acts as a centralized hub that translates and harmonizes access to all these disparate sources. This unified approach not only dramatically reduces integration complexity but also enables more sophisticated cross-platform insights and automation—allowing AI agents to correlate content performance data with audience demographics, automatically tag assets based on distribution metrics, or orchestrate complex workflows that span from content creation to audience engagement analysis.
 
In lab6, we built a simple movie agent assistant that demonstrated MCP servers hosted locally (i.e. stdio transport protocol) as tools to enable the movie assistant agent to answer user questions effectively. In this lab, we'll enhance the solution by creating a unified MCP gateway using Amazon Bedrock AgentCore Gateway, and use the tools provided by the AgentCore gateway to achieve the same goal. 

## Overview
Amazon Bedrock AgentCore Gateway provides an easy and secure way for developers to build, deploy, discover, and connect to tools at scale. AI agents need tools to perform real-world tasks—from querying databases to sending messages to analyzing documents. With Gateway, developers can convert APIs, Lambda functions, and existing services into Model Context Protocol (MCP)-compatible tools and make them available to agents through Gateway endpoints with just a few lines of code. Gateway supports OpenAPI, Smithy, and Lambda as input types, and is the only solution that provides both comprehensive ingress authentication and egress authentication in a fully-managed service. Gateway also provides 1-click integration with several popular tools such as Salesforce, Slack, Jira, Asana, and Zendesk. Gateway eliminates weeks of custom code development, infrastructure provisioning, and security implementation so developers can focus on building innovative agent applications.

In [ ]:
# Make sure you download the latest botocore and boto3 libraries.
import shutil
import subprocess
import sys

def ensure_uv_installed():
    if shutil.which("uv") is None:
        print("🔧 'uv' not found. Installing with pip...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "uv"])
    else:
        print("✅ 'uv' is already installed.")

def uv_install(*packages):
    ensure_uv_installed()
    uv_path = shutil.which("uv")
    print(f"📦 Installing {', '.join(packages)} using uv...")
    subprocess.check_call([uv_path, "pip", "install", *packages])

uv_install("botocore", "boto3")

In [6]:
import boto3
import time
import json
import uuid
import time
import os
import sys

## Restore Variables from Previous Lab Sessions

This cell restores any variables that were stored in previous notebook sessions using the `%store` magic command.


In [7]:
%store -r

## Setup Path and Import Utilities

This cell configures the Python path to import utility modules from the parent directory, including AgentCore utilities and helper functions needed for this lab.


In [3]:
# Get the directory of the current script
if '__file__' in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
else:
    current_dir = os.getcwd()  # Fallback if __file__ is not defined (e.g., Jupyter)

# Navigate to the directory containing utils.py (one level up)
utils_dir = os.path.abspath(os.path.join(current_dir, '..'))

# Add to sys.path
sys.path.insert(0, utils_dir)

# Now you can import utils
import acg_utils
from utils import create_agentcore_role

## Build Lambda Deployment Package

This cell executes the build script that creates a deployment package for the Lambda function. The script packages the required dependencies and code into a zip file for deployment.


In [ ]:
!./build_lambda.sh

## Create AWS Lambda Function for MCP Tools

This cell creates an AWS Lambda function that will serve as the backend for MCP tools. The function is configured with the knowledge base ID from the previous lab and will handle movie-related queries through the AgentCore Gateway.


In [ ]:
#### Create a sample AWS Lambda function that you want to convert into MCP tools
lambda_deployment_package_location_path = "lambda/my_deployment_package.zip"
lambda_function_name = "lab8_lambda_mcp_acg"
lambda_resp = acg_utils.create_gateway_lambda(lambda_deployment_package_location_path, 
                                              lambda_function_name=lambda_function_name, 
                                              envs={ "KB_ID" : lab6_kb_id})

if lambda_resp is not None:
    if lambda_resp['exit_code'] == 0:
        print("Lambda function created with ARN: ", lambda_resp['lambda_function_arn'])
    else:
        print("Lambda function creation failed with message: ", lambda_resp['lambda_function_arn'])

## Create IAM Role for AgentCore Gateway

This cell creates an IAM role that the AgentCore Gateway will assume. This role provides the necessary permissions for the gateway to invoke Lambda functions and manage MCP tools.


In [ ]:
#### Create an IAM role for the Gateway to assume
agentcore_gateway_iam_role = acg_utils.create_agentcore_gateway_role("lab8-lambdagateway")
print("Agentcore gateway role ARN: ", agentcore_gateway_iam_role['Role']['Arn'])

## Setup Cognito Authentication

This cell configures Amazon Cognito for authentication with the AgentCore Gateway. It creates a user pool, resource server, and client credentials that will be used to secure access to the gateway endpoints.


In [ ]:
REGION = os.environ['AWS_DEFAULT_REGION']
USER_POOL_NAME = "lab8-agentcore-gateway-pool"
RESOURCE_SERVER_ID = "lab8-agentcore-gateway-id"
RESOURCE_SERVER_NAME = "lab8-agentcore-gateway-name"
CLIENT_NAME = "lab8-agentcore-gateway-client"
SCOPES = [
    {"ScopeName": "gateway:read", "ScopeDescription": "Read access"},
    {"ScopeName": "gateway:write", "ScopeDescription": "Write access"}
]
scopeString = f"{RESOURCE_SERVER_ID}/gateway:read {RESOURCE_SERVER_ID}/gateway:write"

cognito = boto3.client("cognito-idp", region_name=REGION)

print("Creating or retrieving Cognito resources...")
user_pool_id = acg_utils.get_or_create_user_pool(cognito, USER_POOL_NAME)
print(f"User Pool ID: {user_pool_id}")

acg_utils.get_or_create_resource_server(cognito, user_pool_id, RESOURCE_SERVER_ID, RESOURCE_SERVER_NAME, SCOPES)
print("Resource server ensured.")

client_id, client_secret  = acg_utils.get_or_create_m2m_client(cognito, user_pool_id, CLIENT_NAME, RESOURCE_SERVER_ID)
print(f"Client ID: {client_id}")

# Get discovery URL  
cognito_discovery_url = f'https://cognito-idp.{REGION}.amazonaws.com/{user_pool_id}/.well-known/openid-configuration'
print(cognito_discovery_url)

## Create AgentCore Gateway

This cell creates the Amazon Bedrock AgentCore Gateway with Cognito JWT authorization. The gateway will serve as the MCP server endpoint that agents can connect to for accessing the Lambda-based tools.


In [ ]:
# CreateGateway with Cognito authorizer without CMK. Use the Cognito user pool created in the previous step
gateway_client = boto3.client('bedrock-agentcore-control', region_name = os.environ['AWS_DEFAULT_REGION'])
gateway_name = 'AgentCoreWorkshopGateway'
auth_config = {
    "customJWTAuthorizer": { 
        "allowedClients": [client_id],  # Client MUST match with the ClientId configured in Cognito. Example: 7rfbikfsm51j2fpaggacgng84g
        "discoveryUrl": cognito_discovery_url
    }
}
create_response = gateway_client.create_gateway(name=gateway_name,
    roleArn = agentcore_gateway_iam_role['Role']['Arn'], # The IAM Role must have permissions to create/list/get/delete Gateway 
    protocolType='MCP',
    authorizerType='CUSTOM_JWT',
    authorizerConfiguration=auth_config, 
    description='AgentCore Gateway with AWS Lambda target type'
)
print(create_response)
# Retrieve the GatewayID used for GatewayTarget creation
gatewayID = create_response["gatewayId"]
gatewayURL = create_response["gatewayUrl"]
print(gatewayID)

## Configure Gateway Target with Lambda Backend

This cell creates a gateway target that connects the AgentCore Gateway to the Lambda function. It defines the MCP tool schemas for `get_title_rating` and `get_show_detail` tools that the movie assistant agent will use.


In [14]:
lambda_target_config = {
    "mcp": {
        "lambda": {
            "lambdaArn": lambda_resp['lambda_function_arn'], # Replace this with your AWS Lambda function ARN
            "toolSchema": {
                "inlinePayload": [
                    {
                        "name": "get_title_rating",
                        "description": "tool that retrieves the movie rating for a given title id",
                        "inputSchema": {
                            "type": "object",
                            "properties": {
                                "title_id": {
                                    "type": "string"
                                },

                            },
                            "required": ["title_id"]
                        }
                    },
                    {
                        "name": "get_show_detail",
                        "description": "tool that get the movie or tv show information based on user query",
                        "inputSchema": {
                            "type": "object",
                            "properties": {
                                "query": {
                                    "type": "string"
                                },

                            },
                            "required": ["query"]
                        }
                    }
                ]
            }
        }
    }
}

credential_config = [ 
    {
        "credentialProviderType" : "GATEWAY_IAM_ROLE"
    }
]
targetname='MovieAssistant'
response = gateway_client.create_gateway_target(
    gatewayIdentifier=gatewayID,
    name=targetname,
    description='Lambda Target for Movie Assistant Agent',
    targetConfiguration=lambda_target_config,
    credentialProviderConfigurations=credential_config)

In [ ]:
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name
region

## Store Authentication Configuration

This cell obtains access tokens from Cognito and stores the gateway configuration in AWS Systems Manager Parameter Store and Secrets Manager. This configuration will be used by the MCP client to authenticate with the gateway.


## Get Current AWS Region

This cell retrieves the current AWS region from the boto3 session, which will be used for subsequent AWS service calls and configuration.


In [ ]:
token_response = acg_utils.get_token(user_pool_id, client_id, client_secret,scopeString,REGION)
bearer_access_token = token_response["access_token"]

cognito_config = {
    'pool_id': user_pool_id,
    'client_id': client_id,
    'client_secret' : client_secret,
    'bearer_access_token': bearer_access_token,
    'discovery_url': cognito_discovery_url,
}
# Get discovery URL  
cognito_discovery_url = f'https://cognito-idp.{REGION}.amazonaws.com/{user_pool_id}/.well-known/openid-configuration'
print(cognito_discovery_url)

ssm_client = boto3.client('ssm', region_name=region)
secrets_client = boto3.client('secretsmanager', region_name=region)

try:
    cognito_credentials_response = secrets_client.create_secret(
        Name='mcp_server/cognito/credentials',
        Description='Cognito credentials for MCP server',
        SecretString=json.dumps(cognito_config)
    )
    print("✓ Cognito credentials stored in Secrets Manager")

except secrets_client.exceptions.ResourceExistsException:
    secrets_client.update_secret(
        SecretId='mcp_server/cognito/credentials',
        SecretString=json.dumps(cognito_config)
    )
    print("✓ Cognito credentials updated in Secrets Manager")

agentcore_gateway_url_param_response = ssm_client.put_parameter(
    Name='/mcp_server/gateway/gateway_url',
    Value=gatewayURL,
    Type='String',
    Description='AgentCore Gateway URL for Lambda MCP server',
    Overwrite=True
)
print("\nConfiguration stored successfully!")


## Configure Agent Names and Scripts

This cell sets up the naming convention for the movie assistant agent, including both local and deployed versions, and generates a unique identifier for the agent instance.


In [17]:
agent_name = "movie_assistant_agent_mcp"
agent_local_script = f"{agent_name}_local.py"
agent_script = f"{agent_name}.py"
agent_name = f"{agent_name}_{str(uuid.uuid4())[:5]}"

## Define Test Query

This cell defines a sample query asking about the genre of "Quantum Shadows" and formats it as JSON payload for testing the movie assistant agent.


In [18]:
query = """What is the genre for Quantum Shadows?"""
payload = json.dumps({ "query" : query})

## Test Local Movie Assistant Agent

This cell runs the movie assistant agent locally using the MCP gateway configuration. The `%%time` magic command measures execution time for performance comparison.


In [ ]:
%%time
!python movie_assistant_agent_mcp_local.py '{payload}'

## Create IAM Role for AgentCore Runtime

This cell creates a dedicated IAM role for the AgentCore runtime environment. This role provides the necessary permissions for the deployed agent to access AWS services and the MCP gateway.


In [ ]:
agentcore_iam_role = create_agentcore_role(agent_name=agent_name)

## Configure AgentCore Runtime

This cell configures the AgentCore runtime environment for deploying the movie assistant agent. It specifies the entry point script, execution role, and enables automatic ECR repository creation for containerized deployment.


In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name
region

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint=agent_script,
    execution_role=agentcore_iam_role['Role']['Arn'],
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)
response

## Launch AgentCore Runtime

This cell initiates the deployment of the movie assistant agent to the AgentCore runtime environment. The deployment process includes building the container image and deploying it to AWS infrastructure.


In [ ]:
launch_result = agentcore_runtime.launch()


## Monitor Deployment Status

This cell continuously monitors the deployment status of the AgentCore runtime, polling every 10 seconds until the deployment reaches a final state (READY, CREATE_FAILED, DELETE_FAILED, or UPDATE_FAILED).


In [ ]:
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

## Generate Session ID

This cell creates a unique session ID using UUID4 for tracking conversations with the deployed agent. The session ID maintains context across multiple interactions.


In [25]:
session_id = uuid.uuid4()

## Invoke Deployed Agent

This cell sends the test query to the deployed AgentCore runtime instance, demonstrating how the agent uses the MCP gateway to access movie information through the Lambda-based tools.


In [ ]:
query = """What is the genre for Quantum Shadows?"""
payload = json.dumps({ "query" : query})
invoke_response = agentcore_runtime.invoke(json.loads(payload), session_id=str(session_id))

## Define Output Cleaning Function

This cell defines a utility function to clean and format the agent's response output by removing escape characters, thinking tags, and other formatting artifacts to make the output more readable.


In [27]:
import re
def cleaned_output(invoke_response):
    cleaned_output = "".join([ x.decode("utf-8").replace("\\\"", "") for x in invoke_response["response"]]).replace("\n", "").replace("\"\"", "").replace('\\"', "").replace('\\n', '\n').replace("\"", "")
    cleaned_output = re.sub(r"<thinking>.*?</thinking>", "", cleaned_output, flags=re.DOTALL)
    return cleaned_output

In [ ]:
print(cleaned_output(invoke_response))

## Test Follow-up Query with Context

This cell demonstrates context retention by asking a follow-up question about the movie rating using the same session ID. The agent should understand "this movie" refers to "Quantum Shadows" from the previous query.


In [ ]:
query="What is the rating of this movie?"
payload = json.dumps({ "query" : query})
invoke_response = agentcore_runtime.invoke(json.loads(payload), session_id=str(session_id))
print(cleaned_output(invoke_response))

# Cleanup

In [ ]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

iam_client = boto3.client('iam')

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
    
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

policies = iam_client.list_role_policies(
    RoleName=agentcore_iam_role['Role']['RoleName'],
    MaxItems=100
)

for policy_name in policies['PolicyNames']:
    iam_client.delete_role_policy(
        RoleName=agentcore_iam_role['Role']['RoleName'],
        PolicyName=policy_name
    )
iam_response = iam_client.delete_role(
    RoleName=agentcore_iam_role['Role']['RoleName']
)

acg_utils.delete_gateway(gateway_client,gatewayID)

## Summary

This notebook has successfully demonstrated how to create a unified Model Context Protocol (MCP) gateway using Amazon Bedrock AgentCore Gateway for media and entertainment applications.

### Infrastructure Setup
- **Lambda Function**: Created an AWS Lambda function that serves as the backend for MCP tools, configured with knowledge base access for movie data queries
- **Cognito Authentication**: Set up Amazon Cognito user pool, resource server, and client credentials for secure gateway access

### AgentCore Gateway Configuration
- **Gateway Creation**: Built a production-ready MCP gateway with JWT authorization using Cognito
- **Tool Definition**: Configured two MCP tools (`get_title_rating` and `get_show_detail`) with proper schemas for movie information retrieval
- **Target Integration**: Connected the gateway to the Lambda backend using credential provider configurations

### Agent Development and Testing
- **Local Testing**: Validated the movie assistant agent locally using the MCP gateway configuration
- **Cloud Deployment**: Successfully deployed the agent to AgentCore runtime environment with containerized packaging
- **Functionality Verification**: Tested both individual queries and context-aware follow-up questions demonstrating session management

### Key Benefits Achieved
- **Unified Access**: Created a single MCP endpoint that standardizes access to movie data services
- **Scalability**: Established infrastructure that can easily accommodate additional tools and data sources
- **Security**: Implemented proper authentication and authorization using AWS native services
- **Flexibility**: Demonstrated both local development and cloud deployment scenarios

This lab showcases how MCP gateways can solve the common challenge of siloed data sources in media organizations by providing a unified, secure, and scalable interface for AI agents to access diverse content management systems and analytics platforms.
